In [11]:
import torch
import torchvision.models as models
from PIL import Image


In [12]:
model = models.resnext50_32x4d(pretrained=True)
Image.MAX_IMAGE_PIXELS = None  # or set to a larger value like 933120000

Step 2: Freeze the pre-trained layers

In [13]:
# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

Step 3: Modify the last layer

In [14]:
# Modify the last layer of the model
num_classes = 2 # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)


Step 4: Load the custom dataset

In [15]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(256),
    # transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees = (90,90)),
    transforms.RandomRotation(degrees = (180,180)),
    transforms.RandomRotation(degrees = (270,270)),
    transforms.RandomVerticalFlip(p=1),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and validation datasets
train_dataset = ImageFolder('custom_dataset/train', transform=transform)
val_dataset = ImageFolder('custom_dataset/val', transform=transform)


Step 5: Define the loss function and optimizer

In [16]:
# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)


Step 6: Train the model

In [17]:
from torch.utils.data import DataLoader

# Create data loaders for the train and validation datasets
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [18]:

print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


False


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [19]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move the inputs and labels to the device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_dataset)
        val_acc = running_corrects.double() / len(val_dataset)

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))


Step 7: Fine-tune the model on the custom dataset

In [20]:
# Set the device

# Fine-tune the last layer for a few epochs

optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

# Unfreeze all the layers and fine-tune the entire network for a few more epochs
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/5], train loss: 0.9474, train acc: 0.6027, val loss: 2.3764, val acc: 0.4000
Epoch [2/5], train loss: 0.9178, train acc: 0.7260, val loss: 1.3837, val acc: 0.6400
Epoch [3/5], train loss: 0.7391, train acc: 0.8219, val loss: 1.5434, val acc: 0.7200
Epoch [4/5], train loss: 0.2132, train acc: 0.9178, val loss: 0.0617, val acc: 1.0000
Epoch [5/5], train loss: 1.3367, train acc: 0.8219, val loss: 1.3542, val acc: 0.6400
Epoch [1/10], train loss: 2.2840, train acc: 0.7671, val loss: 2.8562, val acc: 0.9200
Epoch [2/10], train loss: 1.9516, train acc: 0.8219, val loss: 0.4404, val acc: 0.8400
Epoch [3/10], train loss: 0.7306, train acc: 0.8630, val loss: 0.3029, val acc: 0.8800
Epoch [4/10], train loss: 0.9380, train acc: 0.8493, val loss: 0.0804, val acc: 0.9600
Epoch [5/10], train loss: 0.4198, train acc: 0.8767, val loss: 3.0807, val acc: 0.8400
Epoch [6/10], train loss: 0.0731, train acc: 0.9589, val loss: 2.3359, val acc: 0.8800
Epoch [7/10], train loss: 0.1372, train acc: 0.9